### 4-1. 数理最適化問題の解き方を理解する
配送Route を「最適化」するために必要な
1. 問題を創る（内定化）
    - 「総移動距離を最小にする」「総配送Cost を最小にする」など最適化したいものを明確にする。
2. 解く
    大まかに２通りの方法がある。
    - 全探索（確実に正しい答えが導き出せるが、非常に計算時間がかかる。）
    - Heuristic（高速計算が可能だが、厳密な最適解を導き出すことはできない）

という Step を整理する。
